In [48]:

# Define convergence threshold
threshold = 0.001

# Initialize item logits and scores
item_logits = np.array([correct_stills_logodds, correct_walkers_logodds, correct_wave_logodds, correct_walk_logodds])
columns_list = ['is_stills_visited_first', 'is_walkers_visited_last', 'is_wave_command_issued', 'is_walk_command_issued']
assert len(item_logits) == len(columns_list), 'The item logits need to be the same count as the number of columns in scores'
df = item_logits_df[columns_list]
assert df.applymap(lambda x: x not in [0, 1]).sum().sum() == 0, "You have nulls in your data"
scores = df.values

# Find the minimum and maximum values
# min_value = np.min(item_logits)
# max_value = np.max(item_logits)

# Normalize the values
# item_logits = (item_logits - min_value) / (max_value - min_value)

# Define the number of iterations
iterations = 100

In [ ]:

import numpy as np
from scipy.stats import bernoulli

class RaschModel:
    def __init__(self, num_items, num_scenes, scores, item_logits):
        self.num_items = num_items
        self.num_scenes = num_scenes
        self.scores = scores
        self.item_logits = item_logits
        
    def fit(self, iterations, convergence_threshold=0.001):
        person_estimates = np.random.normal(0, 1, size=self.num_scenes)
        print('person_estimates', person_estimates.shape)
        updated_item_logits = self.item_logits.copy()
        
        for _ in range(iterations):
            
            print('updated_item_logits', updated_item_logits.shape, updated_item_logits)
            try:
                print('person_estimates', person_estimates.shape, person_estimates)
                logits = person_estimates + updated_item_logits
            except:
                print('person_estimates[:, np.newaxis]', person_estimates[:, np.newaxis].shape, person_estimates[:, np.newaxis])
                logits = person_estimates[:, np.newaxis] + updated_item_logits
            print('logits', logits.shape, logits)
            
            logits_int = logits.astype(int)
            print('logits_int', logits_int.shape, logits_int)
            
            print('self.scores', self.scores.shape, self.scores)
            predicted_scores = bernoulli.pmf(self.scores, logits_int)
            print('predicted_scores', predicted_scores.shape, predicted_scores)
            
            # Update person estimates using MLE for bernoulli distribution
            person_estimates = np.log(predicted_scores / (1 - predicted_scores))
            print('person_estimates', person_estimates.shape, person_estimates)
            
            # Update item logits based on current person estimates
            updated_item_logits = np.mean(logits, axis=0)
            print()
            print('updated_item_logits', updated_item_logits.shape)
            
            # Check for convergence
            print('self.item_logits', self.item_logits.shape)
            max_diff = np.max(np.abs(updated_item_logits - self.item_logits))
            print('updated_item_logits - self.item_logits', updated_item_logits - self.item_logits)
            print('np.abs(updated_item_logits - self.item_logits)', np.abs(updated_item_logits - self.item_logits))
            print('max_diff', max_diff)
            print('convergence_threshold', convergence_threshold)
            if (max_diff <= convergence_threshold): break

        self.person_estimates = person_estimates
        self.updated_item_logits = updated_item_logits

    def predict(self, new_scores):
        new_logits = self.person_estimates[:, np.newaxis] + self.updated_item_logits
        new_logits_int = new_logits.astype(int)
        return bernoulli.pmf(new_scores, new_logits_int)

# Example usage
num_items = 4
num_scenes = 10
scores = np.random.randint(0, 2, size=(num_scenes, num_items))
item_logits = np.random.randn(num_items)

model = RaschModel(num_items, num_scenes, scores, item_logits)
model.fit(iterations=1000)

person_estimates = model.person_estimates
updated_item_logits = model.updated_item_logits

# Predict scores for new data
new_scores = np.array([[1, 0, 1, 0]])
predicted_probabilities = model.predict(new_scores)

print('Person estimates:', person_estimates)
print('Updated item logits:', updated_item_logits)
print('Predicted probabilities:', predicted_probabilities)

In [30]:

# ### Use Pystan to estimate the item logits iteratively for each scene within a calibration process
def analyze_data(item_logits, scores):
    """
    Performs Rasch analysis using pystan and updates item logits.

    Args:
        item_logits: Current item logits for each scene.
        scores: Scores for each participant on each scene (0 or 1).

    Returns:
        person_estimates: Estimated ability levels for each participant.
        updated_item_logits: Updated item logits for each scene.
    """

    # Stan model code to define the Rasch model (you may need to adjust this based on your specific Rasch model)
    stan_code = """
    data {
        int<lower=0> num_items;  // Number of items (tasks)
        int<lower=0> num_scenes; // Number of participants (scenes)
        matrix[num_scenes, num_items] scores; // Observed responses (0 or 1)
        vector[num_items] item_logits; // Item logits
    }
    
    parameters {
        vector[num_scenes] person_estimates; // Person abilities
    }
    
    model {
        // Rasch model likelihood
        for (i in 1:num_scenes) {
            vector[num_items] logits = item_logits + person_estimates[i];
            int logits_int[num_items];
            for (j in 1:num_items) {
                logits_int[j] = round(logits[j]);
            }
            scores[i] ~ bernoulli_logit(logits_int);
        }
        
        // Prior on person abilities
        person_estimates ~ normal(0, 1);
    }
    """
    
    # Prepare data for Stan model
    data = {
        'num_items': len(item_logits),
        'num_scenes': len(scores),
        'scores': scores,
        'item_logits': item_logits,
    }
    
    # Compile the Stan model
    sm = pystan.StanModel(model_code=stan_code)
    clear_output(wait=True)
    
    # Fit the model with the current item logits
    try: fit = sm.sampling(data=data, init={'beta': item_logits}, iter=1000)
    except ValueError as e:
        print(str(e).strip())
        try: fit = sm.sampling(data=data, init='random', iter=1000)
        
        # Fit the model to the data
        except ValueError as e:
            print(str(e).strip())
            fit = sm.sampling(data=data, iter=1000, chains=4)
    
    # Extract person estimates and updated item logits
    person_estimates = fit['person_estimates']
    updated_item_logits = item_logits + fit['item_logits']
    
    return person_estimates, updated_item_logits

In [31]:

for _ in range(iterations):
    
    # Perform Rasch analysis using the current item logits
    # (This can be done using specialized software or libraries)
    person_estimates, updated_item_logits = analyze_data(item_logits, scores)
    
    # Check for convergence
    if max(abs(updated_item_logits - item_logits)) <= threshold:
        break
    
    # Update item logits for the next iteration
    item_logits = updated_item_logits

ValueError: Failed to parse Stan model 'anon_model_ea35b30a6caf220bd69ab2b9f858089b'. Error message:
SYNTAX ERROR, MESSAGE(S) FROM PARSER:
No matches for: 

  int(real)

Function int not found.
 error in 'unknown file name' at line 19, column 46
  -------------------------------------------------
    17:             int logits_int[num_items];
    18:             for (j in 1:num_items) {
    19:                 logits_int[j] = int(logits[j]);
                                                     ^
    20:             }
  -------------------------------------------------

